# Производительность модели ключ-значение на примере работы с Riak

<p> https://en.wikipedia.org/wiki/Riak
<p> http://riak-python-client.readthedocs.io/en/2.7.0/

```json
kv = {
    'key': [
        "reg_id": <string>,
        "reg_name": <string>,
        "year": <number>,
        "group_id": <number>,
        "group_name": <string>,
        "category": <string>,
        "gender": <string>,
        "value": <number>,
    ]
}
```

In [1]:
! riak version

2.2.3


In [2]:
import riak

In [3]:
client = riak.RiakClient(pb_port=8087)
bucket = client.bucket('crimestatsocial')

## 1. Получаем данные по ключу

In [4]:
bucket.get('1').data

{'reg_id': '1100',
 'reg_name': 'Российская Федерация',
 'year': 2016,
 'group_id': 14002,
 'group_name': 'по возрасту',
 'category': '14-15',
 'gender': 'Мужчины',
 'value': 13573}

In [5]:
%timeit bucket.get('1').data

5.12 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 2. MapReduce на поиск значения по условию "2016 год" из двух ключей

In [6]:
comm = """
    function (v) {
        var parse_data = JSON.parse(v.values[0].data);
        if(parse_data["year"] == 2016)
            result = parse_data;
        else
            result = {};
        return [result];
    }"""
client \
    .map(comm) \
    .add('crimestatsocial', '1') \
    .add('crimestatsocial', '2') \
    .run()

[{'reg_id': '1100',
  'reg_name': 'Российская Федерация',
  'year': 2016,
  'group_id': 14002,
  'group_name': 'по возрасту',
  'category': '16-17',
  'gender': 'Мужчины',
  'value': 29447},
 {'reg_id': '1100',
  'reg_name': 'Российская Федерация',
  'year': 2016,
  'group_id': 14002,
  'group_name': 'по возрасту',
  'category': '14-15',
  'gender': 'Мужчины',
  'value': 13573}]

## 3. MapReduce на подсчет первых 100 значений с "year = 2016"

In [7]:
comm = """
    function (v) {
        var parse_data = JSON.parse(v.values[0].data);
        if(parse_data["year"] == 2016)
            result = 1;
        else
            result = 0;
        return [result];
    }"""
mr = client.map(comm).reduce_sum()
for i in range(100):
    mr.add('crimestatsocial', str(i))
mr.run()

[24]

## 4. Сохранение ключ-значения

In [8]:
data = {
    'reg_id': '1100',
    'reg_name': 'Российская Федерация',
    'year': '2018',
    'group_id': '14002',
    'group_name': 'по возрасту',
    'category': '14-15',
    'gender': 'Мужчины',
    'value': '1'
}
i = int(1e7) - 1
bucket.new(str(i), data=data).store().data

{'reg_id': '1100',
 'reg_name': 'Российская Федерация',
 'year': '2018',
 'group_id': '14002',
 'group_name': 'по возрасту',
 'category': '14-15',
 'gender': 'Мужчины',
 'value': '1'}

In [9]:
i += 1
def foo():
    global i, data
    bucket.new(str(i), data=data).store()
    i += 1

In [10]:
%timeit -n 1000 -r 3 foo()

8.15 ms ± 201 µs per loop (mean ± std. dev. of 3 runs, 1000 loops each)


## 5. Обновление элемента

In [11]:
i = int(1e7) - 1
obj = bucket.get(str(i))
obj.data["year"] = 2017
obj.data["value"] = 1
obj.store().data

{'reg_id': '1100',
 'reg_name': 'Российская Федерация',
 'year': 2017,
 'group_id': '14002',
 'group_name': 'по возрасту',
 'category': '14-15',
 'gender': 'Мужчины',
 'value': 1}

In [12]:
i += 1
def foo():
    global i
    obj = bucket.get(str(i))
    obj.data["year"] = 2017
    obj.data["value"] = 1
    obj.store()
    i += 1

In [13]:
%timeit -n 100 -r 3 foo()

14.4 ms ± 348 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


## 6. Удаление ключа

In [14]:
i = int(1e7) - 1
bucket.delete(str(i)).data

In [15]:
i += 1
def foo():
    global i
    bucket.delete(str(i))
    i += 1

In [16]:
%timeit -n 100 -r 3 foo()

10 ms ± 24.4 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [17]:
client.close()